
#Zelestra X AWS ML Ascend Challenge - Second Edition - Phase 2

In [59]:
%pip install pvlib

##EDA, Feture Engineering, Loss calculation, Theoratical and Actual generation calculations

In [60]:
# Standardized imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pvlib
from datetime import datetime
from typing import Dict, Tuple, Optional

# Constants
PLANT_DC_CAPACITY_KW = 45600
TEMP_COEFF = -0.004
STC_TEMP = 25
INTERVAL_HOURS = 0.25

class SolarPlantAnalysis:
    def __init__(self, filepath: str):
        """Initialize with data loading and validation"""
        self.data = self._load_data(filepath)
        self._validate_data()
        self._clean_data()

    def _load_data(self, filepath: str) -> pd.DataFrame:
        """Load and preprocess data with error handling"""
        try:
            df = pd.read_csv(filepath)
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime').sort_index()
            return df
        except Exception as e:
            raise ValueError(f"Error loading data: {str(e)}")

    def _validate_data(self):
        """Validate critical columns and values"""
        required_cols = ['meteorolgicas_em_03_02_gii', 'inversores_ctin03_inv_03_03_p']
        missing = [col for col in required_cols if col not in self.data.columns]
        if missing:
            raise ValueError(f"Missing required columns: {missing}")

        # Check for negative irradiance values
        irrad_cols = [col for col in self.data.columns if 'gii' in col]
        for col in irrad_cols:
            neg_count = (self.data[col] < 0).sum()
            if neg_count > 0:
                print(f"Warning: {neg_count} negative values in {col}")

    def _clean_data(self):
        """Clean and prepare data"""
        # Drop unnecessary columns
        cols_to_drop = [
            'seguidores_ct03_gcu031_t0308035_workingmode',
            'seguidores_ct08_gcu081_t0808029_workingmode'
        ]
        self.data = self.data.drop(columns=[c for c in cols_to_drop if c in self.data.columns])

        # Fill missing values
        self.data = self.data.fillna(0)

        # Add derived columns
        self.data['hour'] = self.data.index.hour
        self.data['month'] = self.data.index.month

        if 'inverter_id' in self.data.columns:
            self.data['INVERTER'] = self.data['inverter_id']
        elif 'inv_name' in self.data.columns:
            self.data['INVERTER'] = self.data['inv_name']

    def calculate_theoretical_power(self) -> pd.DataFrame:
        """Calculate theoretical power output"""
        # Mean POA irradiance
        poa_cols = [c for c in self.data.columns if "gii" in c and "rear" not in c]
        self.data["G_POA"] = self.data[poa_cols].mean(axis=1)

        # Module temperature
        if "celulas_ctin03_cc_03_1_t_mod" in self.data.columns:
            self.data["T_mod"] = self.data["celulas_ctin03_cc_03_1_t_mod"]
        else:
            amb_cols = [c for c in self.data.columns if "t_amb" in c]
            self.data["T_mod"] = self.data[amb_cols].mean(axis=1)

        # Theoretical power calculation
        self.data["P_theoretical_kW"] = (
            self.data["G_POA"] * PLANT_DC_CAPACITY_KW / 1000 *
            (1 + TEMP_COEFF * (self.data["T_mod"] - STC_TEMP)))
        self.data["P_theoretical_kW"] = self.data["P_theoretical_kW"].clip(lower=0)

        # Theoretical energy
        self.data["E_theoretical_kWh"] = self.data["P_theoretical_kW"] * INTERVAL_HOURS
        self.data["P_theoretical_MW"] = self.data["P_theoretical_kW"] / 1000
        self.data["P_theoretical_energy_MWh"] = self.data["E_theoretical_kWh"] / 1000
        return self.data

    def add_advanced_loss_features(self):
        # Use self.data instead of df
        gamma1 = -0.0045
        gamma2 = -0.0001
        temp_diff = self.data['T_mod_avg'] - 25
        self.data['temp_loss_frac_adv'] = (-gamma1 * temp_diff + gamma2 * temp_diff**2).clip(0)
        self.data['temp_loss_MWh_adv'] = self.data['temp_loss_frac_adv'] * self.data['P_theoretical_energy_MWh']
        # String mismatch model using all string current columns
        string_current_cols = [col for col in df.columns if '_pv_i' in col]
        if string_current_cols:
            df['string_current_mean'] = df[string_current_cols].mean(axis=1)
            df['string_current_std'] = df[string_current_cols].std(axis=1)
            df['string_mismatch_loss_frac'] = (
                df['string_current_std'] / (df['string_current_mean'] + 1e-6)
            ).clip(0, 1)
            df['string_mismatch_loss_MWh'] = df['string_mismatch_loss_frac'] * df['P_theoretical_energy_MWh']
        return df

    def calculate_actual_power(self) -> pd.DataFrame:
        """Calculate actual power from inverters"""
        # Plant-level actual power
        inv_cols = ['inversores_ctin03_inv_03_03_p', 'inversores_ctin08_inv_08_08_p']
        self.data['actual_plant_power_MW'] = self.data[inv_cols].sum(axis=1)

        # Individual inverter power
        self.data['actual_inv1_MW'] = self.data['inversores_ctin03_inv_03_03_p']
        self.data['actual_inv2_MW'] = self.data['inversores_ctin08_inv_08_08_p']

        # Energy calculations (15-min intervals)
        self.data['actual_plant_energy_MWh'] = self.data['actual_plant_power_MW'] * INTERVAL_HOURS
        self.data['actual_inv1_energy_MWh'] = self.data['actual_inv1_MW'] * INTERVAL_HOURS
        self.data['actual_inv2_energy_MWh'] = self.data['actual_inv2_MW'] * INTERVAL_HOURS

        return self.data

    def calculate_performance_ratio(self) -> pd.DataFrame:
        """Calculate performance ratio (PR)"""
        if 'P_theoretical_energy_MWh' not in self.data.columns:
            self.calculate_theoretical_power()

        # Calculate PR (actual/theoretical)
        self.data['PR'] = (self.data['actual_plant_energy_MWh'] /
                          self.data['P_theoretical_energy_MWh']).clip(0, 1.2)
        return self.data

    def calculate_advanced_temp_loss(self):
        """Advanced temperature loss model with quadratic term."""
        gamma1 = -0.0045  # linear term
        gamma2 = -0.0001  # quadratic term (tune as needed)
        temp_diff = self.data['T_mod'] - 25
        self.data['temp_loss_frac_adv'] = (-gamma1 * temp_diff + gamma2 * temp_diff**2).clip(0)
        self.data['temp_loss_MWh_adv'] = self.data['temp_loss_frac_adv'] * self.data['P_theoretical_energy_MWh']

    def calculate_advanced_string_mismatch_loss(self):
        """Data-driven string mismatch loss using string current spread."""
        string_current_cols = [col for col in self.data.columns if '_pv_i' in col]
        if string_current_cols:
            self.data['string_current_mean'] = self.data[string_current_cols].mean(axis=1)
            self.data['string_current_std'] = self.data[string_current_cols].std(axis=1)
            self.data['string_mismatch_loss_frac'] = (
                self.data['string_current_std'] / (self.data['string_current_mean'] + 1e-6)
            ).clip(0, 1)
            self.data['string_mismatch_loss_MWh'] = (
                self.data['string_mismatch_loss_frac'] * self.data['P_theoretical_energy_MWh']
            )
        else:
            self.data['string_current_mean'] = 0
            self.data['string_current_std'] = 0
            self.data['string_mismatch_loss_frac'] = 0
            self.data['string_mismatch_loss_MWh'] = 0

    def calculate_losses(self) -> Dict[str, float]:
        """Calculate different types of losses (includes advanced models)."""
        losses = {}

        # Cloud loss (using clear-sky model)
        if all(col in self.data.columns for col in ['meteorolgicas_em_03_02_ghi', 'meteorolgicas_em_08_01_ghi']):
            self.data['ghi_measured'] = self.data[['meteorolgicas_em_03_02_ghi', 'meteorolgicas_em_08_01_ghi']].mean(axis=1)
            location = pvlib.location.Location(
                latitude=38.0006, longitude=-1.3344,
                tz='Europe/Madrid', altitude=0
            )
            clearsky = location.get_clearsky(self.data.index, model='ineichen')
            self.data['ghi_clearsky'] = clearsky['ghi']
            self.data['cloud_loss_frac'] = (1 - (self.data['ghi_measured'] / self.data['ghi_clearsky'])).clip(0, 1)
            self.data['cloud_loss_MWh'] = self.data['cloud_loss_frac'] * self.data['P_theoretical_energy_MWh']
            losses['cloud_loss'] = self.data['cloud_loss_MWh'].mean()

        # Temperature loss (advanced)
        self.calculate_advanced_temp_loss()
        self.data['temp_loss_MWh'] = self.data['temp_loss_MWh_adv']
        losses['temp_loss'] = self.data['temp_loss_MWh'].mean()


        # Soiling loss (using GII ratio between sensors)
        if all(col in self.data.columns for col in ['meteorolgicas_em_08_01_gii', 'meteorolgicas_em_03_02_gii']):
            self.data['soiling_ratio'] = (
                self.data['meteorolgicas_em_08_01_gii'] /
                self.data['meteorolgicas_em_03_02_gii']
            ).clip(0.7, 1.1)
            self.data['soiling_loss_frac'] = (1 - self.data['soiling_ratio']).clip(0)
            self.data['soiling_loss_MWh'] = self.data['soiling_loss_frac'] * self.data['P_theoretical_energy_MWh']
            losses['soiling_loss'] = self.data['soiling_loss_MWh'].mean()

        # --- Advanced string mismatch loss ---
        self.calculate_advanced_string_mismatch_loss()
        self.data['mismatch_loss_MWh'] = self.data['string_mismatch_loss_MWh']
        losses['mismatch_loss'] = self.data['mismatch_loss_MWh'].mean()

        # Tracker loss (empirical or with data)
        tracker_loss_frac = 0.005
        self.data['tracker_loss_MWh'] = tracker_loss_frac * self.data['P_theoretical_energy_MWh']
        losses['tracker_loss'] = self.data['tracker_loss_MWh'].mean()

        # High wind loss (empirical or with wind speed)
        if 'wind_speed' in self.data.columns:
            stow = self.data['wind_speed'] > 18
            high_wind_loss_frac = 0.01
            self.data['high_wind_loss'] = np.where(stow, high_wind_loss_frac * self.data['P_theoretical_energy_MWh'], 0)
            losses['high_wind_loss'] = self.data['high_wind_loss'].mean()
        else:
            self.data['high_wind_loss'] = 0
            losses['high_wind_loss'] = 0

        # Humidity loss (empirical or with humidity data)
        if 'humidity' in self.data.columns:
            high_humidity = self.data['humidity'] > 90
            humidity_loss_frac = 0.005
            self.data['humidity_loss'] = np.where(high_humidity, humidity_loss_frac * self.data['P_theoretical_energy_MWh'], 0)
            losses['humidity_loss'] = self.data['humidity_loss'].mean()
        else:
            self.data['humidity_loss'] = 0
            losses['humidity_loss'] = 0

        # Inverter loss (fixed 2% of theoretical)
        inverter_loss_frac = 0.02
        self.data['inverter_loss_MWh'] = inverter_loss_frac * self.data['P_theoretical_energy_MWh']
        losses['inverter_loss'] = self.data['inverter_loss_MWh'].mean()

        # DC wiring loss (fixed 1% of theoretical)
        dc_wiring_loss_frac = 0.01
        self.data['dc_wiring_loss_MWh'] = dc_wiring_loss_frac * self.data['P_theoretical_energy_MWh']
        losses['dc_wiring_loss'] = self.data['dc_wiring_loss_MWh'].mean()

        # Shading loss (fixed 1% of theoretical)
        shading_loss_frac = 0.01
        self.data['shading_loss_MWh'] = shading_loss_frac * self.data['P_theoretical_energy_MWh']
        losses['shading_loss'] = self.data['shading_loss_MWh'].mean()


        known_losses = [
            'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
            'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
            'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss'
        ]
        valid_known = [col for col in known_losses if col in self.data.columns]
        self.data['total_known_losses'] = self.data[valid_known].sum(axis=1)
        self.data['unexplained_loss_MWh'] = (
            self.data['P_theoretical_energy_MWh'] -
            self.data['actual_plant_energy_MWh'] -
            self.data['total_known_losses']
        ).clip(0)
        losses['unexplained_loss'] = self.data['unexplained_loss_MWh'].mean()
        return losses


    def plot_comparison(self, freq: str = 'W', show_inverters: bool = False):
        """Plot comparison between theoretical and actual power"""
        resampled = self.data.resample(freq).mean()
        plt.figure(figsize=(15, 6))
        resampled['P_theoretical_MW'].plot(
            label='Theoretical Power', color='black', linestyle='--')
        resampled['actual_plant_power_MW'].plot(
            label='Actual Plant Power', color='green')
        if show_inverters:
            resampled['actual_inv1_MW'].plot(
                label='Inverter 1', color='blue', alpha=0.7)
            resampled['actual_inv2_MW'].plot(
                label='Inverter 2', color='red', alpha=0.7)
        plt.title(f'Theoretical vs Actual Power ({freq} Average)')
        plt.ylabel('Power (MW)')
        plt.xlabel('Date')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_loss_breakdown(self, freq: str = 'W'):
        """Plot stacked area chart of different loss types"""
        loss_cols = [
            'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
            'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
            'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
        ]
        # Ensure all columns exist
        valid_loss_cols = [col for col in loss_cols if col in self.data.columns]
        if not valid_loss_cols:
            print("No loss data available. Run calculate_losses() first.")
            return
        loss_data = self.data[valid_loss_cols].resample(freq).sum()
        plt.figure(figsize=(15, 6))
        ax = loss_data.plot.area(
            stacked=True,
            color=[
                'skyblue', 'orange', 'khaki', 'gray', 'violet', 'gold', 'purple',
                'olive', 'lightgreen', 'lightblue', 'lightcoral'
            ]
        )
        # Add theoretical and actual lines
        theoretical = self.data['P_theoretical_energy_MWh'].resample(freq).sum()
        actual = self.data['actual_plant_energy_MWh'].resample(freq).sum()
        theoretical.plot(ax=ax, color='black', linestyle='--', label='Theoretical')
        actual.plot(ax=ax, color='green', linewidth=2, label='Actual')
        plt.title(f'Energy Loss Breakdown ({freq} Total)')
        plt.ylabel('Energy (MWh)')
        plt.xlabel('Date')
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def analyze_performance_ratio(self):
        """Analyze performance ratio by different conditions"""
        if 'PR' not in self.data.columns:
            self.calculate_performance_ratio()
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        if 'hour' in self.data.columns:
            self.data.groupby('hour')['PR'].mean().plot(
                ax=axes[0,0], marker='o', title='PR by Hour of Day')
            axes[0,0].set_ylim(0, 1.1)
        if 'T_mod' in self.data.columns:
            temp_bins = [0, 20, 40, 60, 80]
            self.data['temp_range'] = pd.cut(self.data['T_mod'], bins=temp_bins)
            self.data.groupby('temp_range')['PR'].mean().plot(
                kind='bar', ax=axes[0,1], title='PR by Temperature Range')
            axes[0,1].set_ylim(0, 1.1)
        if 'PR' in self.data.columns:
            self.data['PR'].hist(ax=axes[1,0], bins=30)
            axes[1,0].set_title('PR Distribution')
            self.data['PR'].resample('W').mean().plot(
                ax=axes[1,1], title='Weekly PR Trend')
            axes[1,1].axhline(0.8, color='red', linestyle='--')
        plt.tight_layout()
        plt.show()
        print("\nPerformance Ratio Statistics:")
        print(f"Mean PR: {self.data['PR'].mean():.3f}")
        print(f"Median PR: {self.data['PR'].median():.3f}")
        print(f"Minimum PR: {self.data['PR'].min():.3f}")
        print(f"Maximum PR: {self.data['PR'].max():.3f}")

    def plot_loss_pie_and_trends(self, freq: str = 'W'):
        """Plot pie chart of average losses and time series of loss trends (highlight unexplained loss)."""
        # Loss columns to include
        loss_cols = [
            'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
            'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
            'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
        ]
        valid_loss_cols = [col for col in loss_cols if col in self.data.columns]

        # Pie chart: average losses
        avg_losses = [self.data[col].mean() for col in valid_loss_cols]

        # Assign colors, with unexplained_loss_MWh as red
        base_colors = [
            'skyblue', 'orange', 'limegreen', 'gray', 'violet', 'gold', 'purple',
            'olive', 'lightgreen', 'lightblue'
        ]
        pie_colors = []
        for col in valid_loss_cols:
            if col == 'unexplained_loss_MWh':
                pie_colors.append('red')
            else:
                # Cycle through base_colors
                pie_colors.append(base_colors.pop(0) if base_colors else 'gray')

        plt.figure(figsize=(8, 6))
        plt.pie(avg_losses, labels=valid_loss_cols, colors=pie_colors, autopct='%1.1f%%', startangle=140)
        plt.title("Average Losses Contribution (Pie Chart)")
        plt.tight_layout()
        plt.show()

        # Time series of each loss (unchanged)
        loss_data = self.data[valid_loss_cols].resample(freq).sum()
        plt.figure(figsize=(15, 6))
        for col in valid_loss_cols:
            plt.plot(loss_data.index, loss_data[col], label=col)
        plt.title(f"Losses Over Time ({freq} totals)")
        plt.ylabel("Loss (MWh)")
        plt.xlabel("Date")
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Correlation heatmap for losses and main metrics
        corr_cols = valid_loss_cols + [
            'G_POA', 'T_mod', 'actual_plant_power_MW', 'PR'
        ]
        corr_cols = [col for col in corr_cols if col in self.data.columns]
        corr = self.data[corr_cols].corr()
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
        plt.title("Correlation Heatmap: Losses and Key Metrics")
        plt.tight_layout()
        plt.show()

    def add_loss_flags(self, threshold_dict=None):
        """
        Adds boolean flag columns for each loss, per interval.
        threshold_dict: dict of {loss_col: threshold}, if None, use mean+std as thresholds
        """
        loss_cols = [
            'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
            'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
            'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
        ]
        for col in loss_cols:
            if col in self.data.columns:
                if threshold_dict and col in threshold_dict:
                    thresh = threshold_dict[col]
                else:
                    # Default: anything above mean+std is flagged
                    thresh = self.data[col].mean() + self.data[col].std()
                self.data[col + '_flag'] = (self.data[col] > thresh).astype(int)

    def get_loss_flags_table(self, cols=None):
        """
        Returns a DataFrame with datetime, asset columns, and boolean loss flags.
        """
        if cols is None:
            cols = ['datetime', 'Zone', 'INVERTER', 'String']  # update as per your data structure
        flag_cols = [c for c in self.data.columns if c.endswith('_flag')]
        out_cols = [c for c in cols if c in self.data.reset_index().columns] + flag_cols
        return self.data.reset_index()[out_cols]

    def aggregate_losses(self, by=['INVERTER'], freq='D'):
        """
        Aggregates losses by asset (e.g., INVERTER, String) and time (e.g., daily).
        Returns a DataFrame ready for reporting (sum of each loss per group).
        """
        loss_cols = [
            'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
            'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
            'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
        ]
        group_cols = []
        if freq:
            group_cols.append(pd.Grouper(freq=freq))
        for asset_col in by:
            if asset_col in self.data.columns:
                group_cols.append(asset_col)
        valid_loss_cols = [col for col in loss_cols if col in self.data.columns]
        agg = self.data.groupby(group_cols)[valid_loss_cols].sum().reset_index()
        return agg

    def asset_ranking(self, loss_type='soiling_loss_MWh', level='INVERTER', top_n=5):
        """
        Ranks assets by mean loss, e.g., best/worst inverter or string.
        """
        if loss_type not in self.data.columns or level not in self.data.columns:
            print(f"{loss_type} or {level} not in data!")
            return None
        ranking = self.data.groupby(level)[loss_type].mean().sort_values(ascending=False)
        print(f"Top {top_n} assets with highest {loss_type}:")
        print(ranking.head(top_n))
        print(f"\nTop {top_n} assets with lowest {loss_type}:")
        print(ranking.tail(top_n))
        return ranking



        # --- Correlation Heatmap: Losses and Key Metrics ---
        # Select columns for correlation (add more as needed)
        corr_cols = valid_loss_cols + [
            'G_POA', 'T_mod', 'actual_plant_power_MW', 'PR'
        ]
        corr_cols = [col for col in corr_cols if col in self.data.columns]
        corr = self.data[corr_cols].corr()

        plt.figure(figsize=(12, 8))
        sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
        plt.title("Correlation Heatmap: Losses and Key Metrics")
        plt.tight_layout()
        plt.show()

    def plot_unexplained_vs_operational(self):
        """Diagnostic plots: unexplained loss vs key operational features"""
        # Example: Temperature
        if 'T_mod_avg' in self.data.columns:
            sns.scatterplot(x=self.data['T_mod_avg'], y=self.data['unexplained_loss_MWh'])
            plt.title('Unexplained Loss vs Module Temperature')
            plt.show()
        # Example: String current std (string mismatch)
        if 'string_current_std' in self.data.columns:
            sns.scatterplot(x=self.data['string_current_std'], y=self.data['unexplained_loss_MWh'])
            plt.title('Unexplained Loss vs String Current Std')
            plt.show()

if __name__ == "__main__":
    try:
        print("Starting solar plant analysis...")
        analysis = SolarPlantAnalysis('/content/drive/MyDrive/Colab_Files/solar hackathon/Dataset 1.csv')
        analysis.calculate_theoretical_power()
        analysis.calculate_actual_power()
        analysis.calculate_performance_ratio()
        losses = analysis.calculate_losses()
        print("\nAverage Losses (MWh per interval):")
        for loss_type, value in losses.items():
            print(f"{loss_type:15s}: {value:.4f}")

        # Existing plots
        analysis.plot_comparison(freq='W', show_inverters=True)
        analysis.plot_loss_breakdown(freq='W')
        analysis.analyze_performance_ratio()
        analysis.plot_loss_pie_and_trends(freq='W')

        # Boolean loss flags for FULL data, export as CSV
        analysis.add_loss_flags()
        flags_table = analysis.get_loss_flags_table()
        flags_table.to_csv("boolean_loss_flags_full.csv", index=False)
        print("Full Boolean Loss Flags Table saved as boolean_loss_flags_full.csv")

        # Aggregated daily losses by inverter (ensure INVERTER column exists!)
        daily_inv_losses = analysis.aggregate_losses(by=['INVERTER'], freq='D')
        print("\nSample Daily Losses by Inverter:")
        print(daily_inv_losses.head())

        # Asset ranking (ensure soiling_loss_MWh and INVERTER exist!)
        analysis.asset_ranking(loss_type='soiling_loss_MWh', level='INVERTER')

        print("\nAnalysis completed successfully!")
    except Exception as e:
        print(f"\nError during analysis: {str(e)}")
        import traceback
        traceback.print_exc()

Starting solar plant analysis...

Average Losses (MWh per interval):
cloud_loss     : 0.5642
temp_loss      : 0.0546
soiling_loss   : 0.1667
mismatch_loss  : 0.3983
tracker_loss   : 0.0085
high_wind_loss : 0.0000
humidity_loss  : 0.0000
inverter_loss  : 0.0339
dc_wiring_loss : 0.0169
shading_loss   : 0.0169
unexplained_loss: 0.5951

Performance Ratio Statistics:
Mean PR: 0.166
Median PR: 0.186
Minimum PR: 0.000
Maximum PR: 1.200
Full Boolean Loss Flags Table saved as boolean_loss_flags_full.csv

Sample Daily Losses by Inverter:
    datetime  cloud_loss_MWh  temp_loss_MWh  soiling_loss_MWh  \
0 2024-10-01       33.953667       8.398847          6.430122   
1 2024-10-02       32.204480       7.343832          7.994864   
2 2024-10-03       33.694271      13.229612         10.215930   
3 2024-10-04       34.253063       9.314687          9.438679   
4 2024-10-05       33.738023       8.611242          6.683591   

   shading_loss_MWh  inverter_loss_MWh  dc_wiring_loss_MWh  mismatch_loss_M

In [61]:
analysis.plot_unexplained_vs_operational()

##Asset ranking Calculations

In [62]:
# Allocate plant-level losses to each inverter
analysis.data['inv1_frac'] = analysis.data['actual_inv1_MW'] / (analysis.data['actual_plant_power_MW'] + 1e-6)
analysis.data['inv2_frac'] = analysis.data['actual_inv2_MW'] / (analysis.data['actual_plant_power_MW'] + 1e-6)

analysis.data['unexplained_loss_inv1_MWh'] = analysis.data['unexplained_loss_MWh'] * analysis.data['inv1_frac']
analysis.data['unexplained_loss_inv2_MWh'] = analysis.data['unexplained_loss_MWh'] * analysis.data['inv2_frac']

# Sum total unexplained loss for each inverter
inv1_total = analysis.data['unexplained_loss_inv1_MWh'].sum()
inv2_total = analysis.data['unexplained_loss_inv2_MWh'].sum()

# Create DataFrame and save as CSV
asset_ranking = pd.DataFrame({
    'INVERTER': ['inv1', 'inv2'],
    'unexplained_loss_MWh': [inv1_total, inv2_total]
}).sort_values('unexplained_loss_MWh', ascending=False)

asset_ranking.to_csv('asset_ranking_unexplained_loss.csv', index=False)
print("Asset ranking saved as asset_ranking_unexplained_loss.csv")

Asset ranking saved as asset_ranking_unexplained_loss.csv


In [63]:
# Save the full analysis.data DataFrame to CSV
analysis.data.to_csv("full_analysis_data.csv")
print("Full analysis DataFrame saved as full_analysis_data.csv")

Full analysis DataFrame saved as full_analysis_data.csv


##Asset ranking by all losses

In [64]:
# List of plant-level losses to allocate and rank
loss_types = [
    'unexplained_loss_MWh',
    'soiling_loss_MWh',
    'mismatch_loss_MWh',
    'temp_loss_MWh',
    'cloud_loss_MWh',
    'tracker_loss_MWh',
    'high_wind_loss',
    'humidity_loss',
    'inverter_loss_MWh',
    'dc_wiring_loss_MWh',
    'shading_loss_MWh'
]



# Compute inverter fractions (if not already done)
analysis.data['inv1_frac'] = analysis.data['actual_inv1_MW'] / (analysis.data['actual_plant_power_MW'] + 1e-6)
analysis.data['inv2_frac'] = analysis.data['actual_inv2_MW'] / (analysis.data['actual_plant_power_MW'] + 1e-6)

# Prepare ranking results
results = {
    'INVERTER': ['inv1', 'inv2']
}

for loss in loss_types:
    inv1_col = f'{loss}_inv1'
    inv2_col = f'{loss}_inv2'
    # Allocate loss proportionally
    analysis.data[inv1_col] = analysis.data[loss] * analysis.data['inv1_frac']
    analysis.data[inv2_col] = analysis.data[loss] * analysis.data['inv2_frac']
    # Sum over all intervals for each inverter
    results[loss] = [
        analysis.data[inv1_col].sum(),
        analysis.data[inv2_col].sum()
    ]

# Create DataFrame and save as CSV
asset_ranking_all = pd.DataFrame(results)
asset_ranking_all.to_csv('asset_ranking_all_losses.csv', index=False)
print("Asset ranking by all losses saved as asset_ranking_all_losses.csv")

Asset ranking by all losses saved as asset_ranking_all_losses.csv


## Total loss breakdown per inverter

In [65]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data (change filename if needed)
asset_ranking = pd.read_csv('asset_ranking_all_losses.csv')


# --- Grouped bar chart ---
ax = asset_ranking.set_index('INVERTER')[loss_types].T.plot(
    kind='bar',
    figsize=(15,7),
    width=0.8
)
plt.title('All Losses by Inverter (Grouped Bar)')
plt.ylabel('Total Loss (MWh)')
plt.xlabel('Loss Type')
plt.xticks(rotation=45)
plt.legend(title='Inverter')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

# --- Stacked bar chart ---
ax = asset_ranking.set_index('INVERTER')[loss_types].plot(
    kind='bar',
    stacked=True,
    figsize=(12, 7)
)
plt.title('Total Loss Breakdown per Inverter (Stacked)')
plt.ylabel('Total Loss (MWh)')
plt.xlabel('Inverter')
plt.xticks(rotation=0)
plt.legend(title='Loss Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

In [66]:
asset_ranking_all.head()

,INVERTER,unexplained_loss_MWh,soiling_loss_MWh,mismatch_loss_MWh,temp_loss_MWh,cloud_loss_MWh,tracker_loss_MWh,high_wind_loss,humidity_loss,inverter_loss_MWh,dc_wiring_loss_MWh,shading_loss_MWh
0,inv1,5181.324969,700.264260,3138.777690,480.098078,2283.299435,72.787167,0.0,0.0,291.148668,145.574334,145.574334
1,inv2,5215.643516,633.957358,3815.905634,474.599931,2527.109087,75.046369,0.0,0.0,300.185475,150.092738,150.092738


## Inverter-level losses, Plant-level losses, Plant-level string mismatch, String mismatch timeseries

In [67]:
import pandas as pd
import re

# Load your main data
df = pd.read_csv("full_analysis_data.csv")

# --- 1. PLANT LEVEL ---
loss_cols = [
    "soiling_loss_MWh", "cloud_loss_MWh", "temp_loss_MWh", "tracker_loss_MWh",
    "inverter_loss_MWh", "dc_wiring_loss_MWh", "shading_loss_MWh", "mismatch_loss_MWh",
    "high_wind_loss", "humidity_loss", "unexplained_loss_MWh"
]

plant_losses = df[loss_cols].sum().reset_index()
plant_losses.columns = ["loss_type", "plant_loss_MWh"]
plant_losses.to_csv("plant_level_energy_losses.csv", index=False)


# --- 2. INVERTER LEVEL ---
inv_loss_cols = [
    "unexplained_loss_inv1_MWh",
    "unexplained_loss_inv2_MWh"
]

inv_agg = []

# Aggregate available inverter loss columns
for col in inv_loss_cols:
    if col in df.columns:
        inv_id = col.split("_")[2]  # e.g. 'inv1', 'inv2'
        total = df[col].sum()
        inv_agg.append({
            "inverter": inv_id,
            "loss_type": "unexplained_loss",
            "loss_MWh": total
        })

# Also aggregate inverter actual energy if useful
inv_energy_cols = [
    "actual_inv1_energy_MWh",
    "actual_inv2_energy_MWh"
]
for col in inv_energy_cols:
    if col in df.columns:
        inv_id = col.split("_")[2]  # e.g. 'inv1', 'inv2'
        total = df[col].sum()
        inv_agg.append({
            "inverter": inv_id,
            "loss_type": "actual_energy",
            "loss_MWh": total
        })

if inv_agg:
    inverter_losses = pd.DataFrame(inv_agg)
    inverter_losses.to_csv("inverter_level_energy_losses.csv", index=False)
    print("Inverter-level losses saved as inverter_level_energy_losses.csv")
else:
    print("No inverter-level loss columns found!")


# --- 3. STRING MISMATCH (AGGREGATE & TIME SERIES) ---
# These are not per-string but describe string mismatch for the plant as a whole.

string_mismatch_loss_col = "string_mismatch_loss_MWh"
string_mismatch_frac_col = "string_mismatch_loss_frac"
string_current_mean_col = "string_current_mean"
string_current_std_col = "string_current_std"

agg_string_losses = {
    "total_string_mismatch_loss_MWh": df[string_mismatch_loss_col].sum() if string_mismatch_loss_col in df.columns else None,
    "avg_string_current_std": df[string_current_std_col].mean() if string_current_std_col in df.columns else None,
    "max_string_current_std": df[string_current_std_col].max() if string_current_std_col in df.columns else None,
    "avg_string_mismatch_loss_frac": df[string_mismatch_frac_col].mean() if string_mismatch_frac_col in df.columns else None,
}
string_mismatch_summary = pd.DataFrame([agg_string_losses])
string_mismatch_summary.to_csv("string_mismatch_plant_summary.csv", index=False)

# Save time series for string mismatch loss and current std (optional for further analysis)
string_ts_cols = []
for col in [string_mismatch_loss_col, string_current_std_col, string_current_mean_col, string_mismatch_frac_col]:
    if col in df.columns:
        string_ts_cols.append(col)

# Try to add a time column if present
for tcol in ["datetime", "hour", "timestamp"]:
    if tcol in df.columns:
        time_col = tcol
        break
else:
    time_col = None

if time_col:
    string_ts_cols = [time_col] + string_ts_cols

if string_ts_cols:
    df[string_ts_cols].to_csv("string_mismatch_timeseries.csv", index=False)

# --- 4. STRING LEVEL (if per-string columns exist) ---
# For demonstration, search for all columns ending in "_stringXX"
string_loss_pattern = re.compile(r'(.+)_string(\d+)$')
string_loss_entries = []

for col in df.columns:
    match = string_loss_pattern.match(col)
    if match:
        loss_type, string_id = match.groups()
        total = df[col].sum()
        string_loss_entries.append({"string": string_id, "loss_type": loss_type, "loss_MWh": total})

if string_loss_entries:
    string_losses = pd.DataFrame(string_loss_entries)
    string_losses.to_csv("string_level_energy_losses.csv", index=False)
    print("String-level losses saved as string_level_energy_losses.csv")
# else:
#     print("No string-level loss columns found.")

# --- 5. Print summary ---
print("Plant-level losses saved as plant_level_energy_losses.csv")
print("Inverter-level losses saved as inverter_level_energy_losses.csv")
print("Plant-level string mismatch summary saved as string_mismatch_plant_summary.csv")
if string_ts_cols:
    print("String mismatch timeseries saved as string_mismatch_timeseries.csv")

Inverter-level losses saved as inverter_level_energy_losses.csv
Plant-level losses saved as plant_level_energy_losses.csv
Inverter-level losses saved as inverter_level_energy_losses.csv
Plant-level string mismatch summary saved as string_mismatch_plant_summary.csv
String mismatch timeseries saved as string_mismatch_timeseries.csv


##Loss boolean flags

In [68]:

flag_cols = ['cloud_loss_MWh', 'shading_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'unexplained_loss_MWh']
threshold = 0.01  # set your threshold

flags = pd.DataFrame()
flags['datetime'] = df['datetime']
for col in flag_cols:
    if col in df.columns:
        flags[col.replace('_MWh', '') + '_flag'] = (df[col] > threshold).astype(int)

flags.to_csv('loss_boolean_flags_15min.csv', index=False)

##Daily, Weekly and Monthly loss calculation

In [69]:


# Assuming df is loaded from "full_analysis_data.csv"
# Convert 'datetime' column to datetime objects
df['datetime'] = pd.to_datetime(df['datetime'])

daily_losses = df.groupby(df['datetime'].dt.date)[loss_cols].sum()
daily_losses.to_csv('losses_daily.csv')

# Weekly
df['week'] = df['datetime'].dt.to_period('W').astype(str)
weekly_losses = df.groupby('week')[loss_cols].sum()
weekly_losses.to_csv('losses_weekly.csv')

# Monthly
df['month_period'] = df['datetime'].dt.to_period('M').astype(str)
monthly_losses = df.groupby('month_period')[loss_cols].sum()
monthly_losses.to_csv('losses_monthly.csv')

##Estimate of per sting currents

In [70]:


# Select all string current columns
string_current_cols = [col for col in df.columns if '_pv_i' in col]

# Get statistics per string
string_stats = df[string_current_cols].agg(['mean', 'std', 'min', 'max']).T.reset_index()
string_stats.columns = ['string_column', 'mean', 'std', 'min', 'max']
string_stats.to_csv('string_current_stats.csv', index=False)

##Estimation of String-Level Mismatch Loss Allocation Based on Current Imbalance

In [71]:
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])

string_current_cols = [col for col in df.columns if '_pv_i' in col]  # all string current columns

per_string_loss_entries = []

for idx, row in df.iterrows():
    currents = row[string_current_cols].values.astype(float)
    mean_current = np.mean(currents)
    deviations = np.abs(currents - mean_current)
    total_deviation = deviations.sum()
    if total_deviation > 0:
        allocations = deviations / total_deviation
    else:
        allocations = np.zeros_like(deviations)
    # Get the plant-level mismatch loss for this interval
    mismatch_loss = 0
    if 'string_mismatch_loss_MWh' in row and not pd.isna(row['string_mismatch_loss_MWh']):
        mismatch_loss = row['string_mismatch_loss_MWh']
    elif 'mismatch_loss_MWh' in row and not pd.isna(row['mismatch_loss_MWh']):
        mismatch_loss = row['mismatch_loss_MWh']
    # Allocate loss per string
    per_string_loss = allocations * mismatch_loss
    for i, col in enumerate(string_current_cols):
        per_string_loss_entries.append({
            'datetime': row['datetime'] if 'datetime' in row else idx,
            'string_id': col,
            'estimated_mismatch_loss_MWh': per_string_loss[i]
        })

per_string_loss_df = pd.DataFrame(per_string_loss_entries)
per_string_loss_df.to_csv("per_string_mismatch_loss_timeseries.csv", index=False)
print("Saved per-string mismatch loss time series to per_string_mismatch_loss_timeseries.csv")

Saved per-string mismatch loss time series to per_string_mismatch_loss_timeseries.csv


##Prediction of Theoretical and Actual Generation and Comparison with Corresponding Calculated Values

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# Load data
df = pd.read_csv("full_analysis_data.csv")

# --- 1. Define features and targets ---
# Theoretical generation model
features_theoretical = [
    "G_POA", "T_mod", "ghi_measured", "ghi_clearsky", "month", "hour"
]
target_theoretical = "E_theoretical_kWh"

# Actual generation model
features_actual = features_theoretical + [
    "soiling_loss_MWh", "cloud_loss_MWh", "temp_loss_MWh", "tracker_loss_MWh",
    "inverter_loss_MWh", "dc_wiring_loss_MWh", "shading_loss_MWh", "mismatch_loss_MWh"
]
target_actual = "actual_plant_energy_MWh"

# --- 2. Drop rows with missing values in used columns
df_theo = df[features_theoretical + [target_theoretical]].dropna()
df_act = df[features_actual + [target_actual]].dropna()

# --- 3. Split data ---
X_train_theo, X_test_theo, y_train_theo, y_test_theo = train_test_split(
    df_theo[features_theoretical], df_theo[target_theoretical], test_size=0.2, random_state=42
)
X_train_act, X_test_act, y_train_act, y_test_act = train_test_split(
    df_act[features_actual], df_act[target_actual], test_size=0.2, random_state=42
)

# --- 4. Train models ---
model_theo = RandomForestRegressor(n_estimators=100, random_state=42)
model_theo.fit(X_train_theo, y_train_theo)
pred_theo = model_theo.predict(X_test_theo)

model_act = RandomForestRegressor(n_estimators=100, random_state=42)
model_act.fit(X_train_act, y_train_act)
pred_act = model_act.predict(X_test_act)

# --- 5. Evaluate models ---
print("Theoretical Gen Model: R2 =", r2_score(y_test_theo, pred_theo), "MAE =", mean_absolute_error(y_test_theo, pred_theo))
print("Actual Gen Model:      R2 =", r2_score(y_test_act, pred_act), "MAE =", mean_absolute_error(y_test_act, pred_act))

# --- 6. Compare with calculated values and plot ---
# Theoretical Generation: predicted vs. calculated
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(y_test_theo, pred_theo, alpha=0.5, label='Prediction')
plt.plot([y_test_theo.min(), y_test_theo.max()], [y_test_theo.min(), y_test_theo.max()], 'r--', label='Ideal')
plt.xlabel('Calculated Theoretical (kWh)')
plt.ylabel('Predicted Theoretical (kWh)')
plt.title('Theoretical Generation\nPrediction vs. Actual')
plt.legend()

# Actual Generation: predicted vs. calculated
plt.subplot(1,2,2)
plt.scatter(y_test_act, pred_act, alpha=0.5, label='Prediction')
plt.plot([y_test_act.min(), y_test_act.max()], [y_test_act.min(), y_test_act.max()], 'r--', label='Ideal')
plt.xlabel('Calculated Actual (MWh)')
plt.ylabel('Predicted Actual (MWh)')
plt.title('Actual Generation\nPrediction vs. Actual')
plt.legend()
plt.tight_layout()
plt.show()

# --- 7. Time series comparison for a sample (if you have a time column, e.g. 'hour' or 'datetime') ---
# You can plot a slice by index or by time
if "hour" in df.columns:
    # plt.figure(figsize=(12,4))
    # idx = X_test_theo.index[:100]
    # plt.plot(df.loc[idx, "hour"], y_test_theo.loc[idx], label="Theoretical Actual")
    # plt.plot(df.loc[idx, "hour"], pred_theo[:100], label="Theoretical Predicted")
    # plt.plot(df.loc[idx, "hour"], y_test_act.loc[idx], label="Actual Actual")
    # plt.plot(df.loc[idx, "hour"], pred_act[:100], label="Actual Predicted")
    # plt.xlabel("Hour")
    # plt.ylabel("Generation")
    # plt.title("Generation Comparison (Sample 100)")
    # plt.legend()
    # plt.show()
    # Option 1: Use .iloc to plot the first 100 samples from each test set
    plt.figure(figsize=(12,4))
    n = min(100, len(y_test_theo), len(y_test_act))  # in case test sets are small

    # Theoretical
    plt.plot(range(n), y_test_theo.iloc[:n], label="Theoretical Actual")
    plt.plot(range(n), pred_theo[:n], label="Theoretical Predicted")

    # Actual
    plt.plot(range(n), y_test_act.iloc[:n], label="Actual Actual")
    plt.plot(range(n), pred_act[:n], label="Actual Predicted")

    plt.xlabel("Sample Index")
    plt.ylabel("Generation")
    plt.title("Generation Comparison (First {} Test Samples)".format(n))
    plt.legend()
    plt.show()

Theoretical Gen Model: R2 = 0.99995191491369 MAE = 6.42958575696794
Actual Gen Model:      R2 = 0.9367899771356037 MAE = 0.05328755508849305


##Daily loss breakdown

In [73]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("full_analysis_data.csv", parse_dates=['datetime'])
loss_cols = ['cloud_loss_MWh', 'shading_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'unexplained_loss_MWh']

# Stacked bar of daily loss breakdown
daily_losses = df.groupby(df['datetime'].dt.date)[loss_cols].sum()
daily_losses.plot(kind='bar', stacked=True, figsize=(14,6))
plt.title('Daily Loss Breakdown by Class')
plt.ylabel('Energy Loss (MWh)')
plt.xlabel('Date')
plt.tight_layout()
plt.savefig('daily_loss_breakdown.png')
plt.show()

# Asset ranking (e.g., by total unexplained loss, if inverter-level data exists)
if 'unexplained_loss_inv1_MWh' in df.columns and 'unexplained_loss_inv2_MWh' in df.columns:
    asset_ranking = {
        'inv1': df['unexplained_loss_inv1_MWh'].sum(),
        'inv2': df['unexplained_loss_inv2_MWh'].sum()
    }
    pd.Series(asset_ranking).sort_values(ascending=False).plot(kind='bar')
    plt.title('Inverter Asset Ranking by Unexplained Loss')
    plt.ylabel('Total Unexplained Loss (MWh)')
    plt.tight_layout()
    plt.savefig('inverter_asset_ranking.png')
    plt.show()
    plt.close()

##Final loss summary

In [74]:
loss_cols = [
    'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
    'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
    'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
]
summary = df[loss_cols].sum().sort_values(ascending=False)
summary.to_csv('final_loss_summary.csv')

##Summary of the tables generated

In [75]:
import os

csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print("CSV files in current directory:")
for f in csv_files:
    print(f)

CSV files in current directory:
plant_level_energy_losses.csv
asset_ranking_unexplained_loss.csv
per_string_mismatch_loss_timeseries.csv
losses_daily.csv
inverter_level_energy_losses.csv
asset_ranking_all_losses.csv
string_current_stats.csv
string_mismatch_timeseries.csv
string_mismatch_plant_summary.csv
final_loss_summary.csv
loss_boolean_flags_15min.csv
losses_monthly.csv
losses_weekly.csv
full_analysis_data.csv
boolean_loss_flags_full.csv


#Dashboard generation

In [76]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your main data
df = pd.read_csv("full_analysis_data.csv", parse_dates=['datetime'])

# Update this list to only include present columns and keep order:
loss_cols = [
    'cloud_loss_MWh', 'temp_loss_MWh', 'soiling_loss_MWh', 'shading_loss_MWh',
    'inverter_loss_MWh', 'dc_wiring_loss_MWh', 'mismatch_loss_MWh',
    'tracker_loss_MWh', 'high_wind_loss', 'humidity_loss', 'unexplained_loss_MWh'
]
loss_cols = [col for col in loss_cols if col in df.columns]

# Asset columns
asset_loss_cols = [col for col in df.columns if ("_inv" in col or "_string" in col) and col.endswith("_MWh")]

def loss_pie_chart():
    fig, ax = plt.subplots()
    loss_sum = df[loss_cols].sum()
    colors = ['skyblue', 'orange', 'limegreen', 'gray', 'violet', 'gold', 'purple',
              'olive', 'lightgreen', 'lightblue', 'red']
    ax.pie(loss_sum, labels=loss_cols, colors=colors[:len(loss_cols)], autopct='%1.1f%%')
    ax.set_title("Loss Breakdown (Pie Chart)")
    return fig

def losses_over_time(freq):
    fig, ax = plt.subplots(figsize=(11, 5))
    df2 = df.set_index('datetime')[loss_cols].resample(freq).sum()
    df2.plot(ax=ax)
    ax.set_title(f"Losses Over Time ({freq})")
    ax.set_ylabel("Loss (MWh)")
    ax.set_xlabel("Date")
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    return fig

def performance_ratio_analysis():
    import pandas as pd
    import matplotlib.pyplot as plt

    # Work on a copy
    dfx = df.copy()

    # Make sure PR exists
    if 'PR' not in dfx.columns:
        # Try to calculate if possible, else display error
        if 'actual_plant_energy_MWh' in dfx.columns and 'P_theoretical_energy_MWh' in dfx.columns:
            dfx['PR'] = (dfx['actual_plant_energy_MWh'] / dfx['P_theoretical_energy_MWh']).clip(0, 1.2)
        else:
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, 'Missing PR and energy columns', ha='center', va='center')
            return fig

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # PR by hour of day
    if 'hour' in dfx.columns:
        pr_hour = dfx.groupby('hour')['PR'].mean()
        axes[0,0].plot(pr_hour.index, pr_hour.values, marker='o')
        axes[0,0].set_title('PR by Hour of Day')
        axes[0,0].set_ylim(0, 1.1)
        axes[0,0].set_xlabel('Hour')
        axes[0,0].set_ylabel('PR')
    else:
        axes[0,0].text(0.5, 0.5, 'No hour data', ha='center', va='center')

    # PR by temperature range
    if 'T_mod' in dfx.columns:
        temp_bins = [0, 20, 40, 60, 80]
        temp_range = pd.cut(dfx['T_mod'], bins=temp_bins)
        dfx2 = dfx.copy()
        dfx2['temp_range'] = temp_range
        pr_temp = dfx2.groupby('temp_range')['PR'].mean()
        axes[0,1].bar([str(x) for x in pr_temp.index], pr_temp.values)
        axes[0,1].set_title('PR by Temperature Range')
        axes[0,1].set_ylim(0, 1.1)
        axes[0,1].set_xlabel('Temperature Range')
        axes[0,1].set_ylabel('PR')
    else:
        axes[0,1].text(0.5, 0.5, 'No T_mod data', ha='center', va='center')

    # PR histogram
    dfx['PR'].dropna().hist(ax=axes[1,0], bins=30)
    axes[1,0].set_title('PR Distribution')
    axes[1,0].set_xlabel('PR')
    axes[1,0].set_ylabel('Frequency')

    # PR trend over time
    if 'datetime' in dfx.columns and pd.api.types.is_datetime64_any_dtype(dfx['datetime']):
        dfx_time = dfx.set_index('datetime')
        pr_trend = dfx_time['PR'].resample('W').mean()
        axes[1,1].plot(pr_trend.index, pr_trend.values)
        axes[1,1].set_title('Weekly PR Trend')
        axes[1,1].axhline(0.8, color='red', linestyle='--')
        axes[1,1].set_xlabel('Date')
        axes[1,1].set_ylabel('PR')
    else:
        axes[1,1].text(0.5, 0.5, 'No datetime data', ha='center', va='center')

    plt.tight_layout()
    return fig
def asset_ranking(col):
    if col not in df.columns:
        return f"Column {col} not found in data."
    ranking = df[col]
    fig, ax = plt.subplots(figsize=(9, 4))
    ranking.plot(kind='hist', ax=ax, bins=30)
    ax.set_title(f"Distribution: {col}")
    ax.set_ylabel("Frequency")
    ax.set_xlabel(col)
    plt.tight_layout()
    return fig

def loss_flags_table():
    flag_cols = [col for col in df.columns if col.endswith('_flag')]
    if not flag_cols:
        return pd.DataFrame({'info': ['No flag columns found.']})
    cols = ['datetime'] + [c for c in ['Zone', 'INVERTER', 'String'] if c in df.columns] + flag_cols
    out = df[cols].copy()
    return out.head(100)  # Only show first 100 rows for brevity

def correlation_heatmap():
    corr_cols = loss_cols + [c for c in ['G_POA', 'T_mod', 'actual_plant_power_MW', 'PR'] if c in df.columns]
    corr = df[corr_cols].corr()
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
    ax.set_title("Correlation Heatmap: Losses and Key Metrics")
    plt.tight_layout()
    return fig

with gr.Blocks() as demo:
    gr.Markdown("# ☀️ Solar PV Loss Attribution Dashboard")
    gr.Markdown("Explore and interact with your plant's loss data. Select a visualization below.")
    with gr.Tab("Loss Breakdown Pie"):
        gr.Plot(loss_pie_chart)
    with gr.Tab("Losses Over Time"):
        freq = gr.Radio(['W', 'D', 'M'], label="Frequency", value='W')
        gr.Plot(losses_over_time, inputs=[freq])
    with gr.Tab("Performance Ratio Analysis"):
        gr.Plot(performance_ratio_analysis)
    with gr.Tab("Asset Loss Distribution"):
        asset_col = gr.Dropdown(asset_loss_cols, label="Asset loss column", value=asset_loss_cols[0] if asset_loss_cols else None)
        gr.Plot(asset_ranking, inputs=[asset_col])
    with gr.Tab("Loss Flags Table"):
        gr.Dataframe(loss_flags_table)
    with gr.Tab("Correlation Heatmap"):
        gr.Plot(correlation_heatmap)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52a4c67a7533b328b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##LLM-powered executive report

In [77]:
import pandas as pd
import openai
import os

# --- 1. Load all CSVs you have ---
csv_paths = {
    'full_analysis': 'full_analysis_data.csv',
    'plant_loss': 'plant_level_energy_losses.csv',
    'inverter_loss': 'inverter_level_energy_losses.csv',
    'string_mismatch_summary': 'string_mismatch_plant_summary.csv',
    'string_mismatch_timeseries': 'string_mismatch_timeseries.csv',
    'per_string_mismatch_timeseries': 'per_string_mismatch_loss_timeseries.csv',
    'string_current_stats': 'string_current_stats.csv',
    'bool_flags_full': 'boolean_loss_flags_full.csv',
    'loss_flags_15min': 'loss_boolean_flags_15min.csv',
    'final_loss_summary': 'final_loss_summary.csv',
    'losses_weekly': 'losses_weekly.csv',
    'losses_monthly': 'losses_monthly.csv',
    'losses_daily': 'losses_daily.csv',
    'asset_ranking_all': 'asset_ranking_all_losses.csv',
    'asset_ranking_unexpl': 'asset_ranking_unexplained_loss.csv'
}

dfs = {k: pd.read_csv(v) for k, v in csv_paths.items() if os.path.exists(v)}

# --- 2. Key columns (update if needed) ---
datetime_col = "hour"
gen_col = "actual_plant_energy_MWh"
theoretical_col = "P_theoretical_energy_MWh"
pr_col = "PR"
unexpl_col = "unexplained_loss_MWh"
soiling_col = "soiling_loss_MWh"
cloud_col = "cloud_loss_MWh"
temp_col = "temp_loss_MWh"
tracker_col = "tracker_loss_MWh"
inv_loss_col = "inverter_loss_MWh"
dc_loss_col = "dc_wiring_loss_MWh"
shading_col = "shading_loss_MWh"
mismatch_col = "mismatch_loss_MWh"

# --- 3. Compute summary statistics from main data ---
df_full = dfs['full_analysis']
summary_text = f"""
Period: {df_full[datetime_col].min()} to {df_full[datetime_col].max()}
Total Actual Generation: {df_full[gen_col].sum():.2f} MWh
Theoretical Generation: {df_full[theoretical_col].sum():.2f} MWh
Performance Ratio (avg): {df_full[pr_col].mean():.3f}
Known Losses (MWh):
  - Soiling: {df_full[soiling_col].sum():.2f}
  - Cloud: {df_full[cloud_col].sum():.2f}
  - Temp: {df_full[temp_col].sum():.2f}
  - Tracker: {df_full[tracker_col].sum():.2f}
  - Inverter: {df_full[inv_loss_col].sum():.2f}
  - DC Wiring: {df_full[dc_loss_col].sum():.2f}
  - Shading: {df_full[shading_col].sum():.2f}
  - String Mismatch: {df_full[mismatch_col].sum():.2f}
Unexplained Loss (total): {df_full[unexpl_col].sum():.2f} MWh ({(df_full[unexpl_col].sum()/df_full[theoretical_col].sum()*100):.1f}% of theoretical)
Highest unexplained loss (interval): {df_full[unexpl_col].max():.2f} MWh at {df_full.loc[df_full[unexpl_col].idxmax(), datetime_col]}
"""

# --- 4. Top assets by total loss and unexplained loss ---
all_losses = dfs['asset_ranking_all']
unexpl = dfs['asset_ranking_unexpl']
summary_text += f"\nTop 5 assets by total loss:\n{all_losses.head(5).to_string(index=False)}\n"
summary_text += f"\nTop 5 assets by unexplained loss:\n{unexpl.head(5).to_string(index=False)}\n"

# --- 5. Add event statistics from full boolean flags table ---
bool_flags = dfs['bool_flags_full']
flag_sums = bool_flags.sum(numeric_only=True)
summary_text += f"\nLoss event counts (boolean flags, all assets summed):\n{flag_sums.to_string()}\n"

# --- 6. Add more summaries from other files as desired ---
if 'plant_loss' in dfs:
    plant_loss_summary = dfs['plant_loss'].describe().to_string()
    summary_text += f"\nPlant-level loss summary:\n{plant_loss_summary}\n"
if 'final_loss_summary' in dfs:
    summary_text += f"\nFinal loss summary:\n{dfs['final_loss_summary'].to_string(index=False)}\n"
if 'string_mismatch_summary' in dfs:
    summary_text += f"\nString mismatch plant summary (top 5):\n{dfs['string_mismatch_summary'].head(5).to_string(index=False)}\n"
if 'string_current_stats' in dfs:
    summary_text += f"\nString current stats (top 5):\n{dfs['string_current_stats'].head(5).to_string(index=False)}\n"

# --- 7. Prepare the LLM prompt ---


prompt = f"""
You are a solar plant analyst.

Given the following comprehensive summary from all plant CSVs, generate a clear, concise executive report with the following deliverables:

DELIVERABLES:
• A report/dashboard with:
  - Theoretical generation (model-based) summary.
  - Actual vs. theoretical generation comparison (numerical values, visual/graphical description).
  - Boolean flags for each 15-min interval and each loss class (show sample table if space is limited).
  - Quantified losses by class (soiling, cloud, temperature, tracker, inverter, DC wiring, shading, string mismatch, unexplained), for 15-min, hourly, daily, weekly, monthly, and at Plant, Inverter, and String levels. Add insights and reasoning. Use tables as needed.
  - Asset ranking: best-performing, under-performing, high/medium soiling loss, and other significant loss categories.
  - Table attributing losses for each asset (Asset Name, Loss Class, Total Loss MWh).
  - Organize the report clearly with section headings and tables.
  - Highlight major findings in simple, executive-friendly language.
  - Identify any assets that need urgent O&M attention.
  - Explain large unexplained losses and their possible causes.
  - Give at least 2 actionable recommendations for plant improvement.

Summary:
{summary_text}
"""

# --- 8. OpenAI API key (set securely!) ---
openai_api_key = os.environ.get("OPENAI_API_KEY", '')  # Or set manually, but don't commit to code

def generate_llm_report(prompt, openai_api_key):
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2500,
        temperature=0.4
    )
    return response.choices[0].message.content
with open("solar_plant_report.md", "w", encoding="utf-8") as f:
    f.write(report)

# --- 9. Run and print report ---
if __name__ == "__main__":
    print("Generating LLM-powered executive report...\n")
    if not openai_api_key:
        print("OpenAI API key not found. Please set your API key in the environment variable 'OPENAI_API_KEY'.")
    else:
        report = generate_llm_report(prompt, openai_api_key)
        print("----- Solar Plant Executive Report -----\n")
        print(report)

Generating LLM-powered executive report...

----- Solar Plant Executive Report -----

# Executive Solar Plant Performance Report  
**Period:** 0 to 23 hours  
**Plant:** [Name/ID withheld]  

---

## 1. Theoretical Generation Summary  
- **Total Theoretical Generation:** 29,572.16 MWh  
- This value represents the modeled maximum expected output under ideal conditions, accounting for irradiance, temperature, and system design parameters.

---

## 2. Actual vs. Theoretical Generation Comparison  

| Metric                  | Value (MWh) | % of Theoretical |
|-------------------------|-------------|------------------|
| Actual Generation       | 5,011.90    | 16.96%           |
| Theoretical Generation  | 29,572.16   | 100%             |
| Performance Ratio (Avg) | 0.166       | 16.6%            |

**Insight:**  
The plant operated at only ~17% of its theoretical capacity, indicating significant losses and underperformance.

**Visual Description:**  
- A bar chart would show a tall bar f

Thanks